# Run H2O on enterprise cluster
Assumes you have launched the H2O cluster from Enteprise Steam UI in H2O AI Cloud https://h2o.ai/freetrial/

In [ ]:
import h2o
import h2osteam
from h2osteam.clients import H2oKubernetesClient 

## replace url, username, password, cluster-name with your specifics
conn = h2osteam.login(
    url="https://steam.cloud.h2o.ai",
    verify_ssl=False,
    username="ourH2OAICloudUserName",
    password="yourH2OAICloudPassword-or-PAT")


In [ ]:
cluster = H2oKubernetesClient().get_cluster("test_2", created_by="greg.keys@oracle.com") 
cluster.connect()

## Import the data 

In [ ]:
input_csv = "https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-at-Scale-with-H2O/main/chapt3/loans-lite.csv"
loans = h2o.import_file(input_csv)
loans.dim

In [ ]:
loans.head()

## Clean the data

In [ ]:
loans["bad_loan"] = loans["bad_loan"].asfactor()

## Engineer new features from the original data

In [ ]:
loans["issue_d_year"] = loans["issue_d"].year().asfactor()
loans["issue_d_month"] = loans["issue_d"].month().asfactor()

## Model training

In [ ]:
train, validate, test = loans.split_frame(seed=1, ratios=[0.7, 0.15])

In [ ]:
predictors = list(loans.col_names)
predictors.remove("bad_loan")
predictors.remove("issue_d")

In [ ]:
# Note: at the time of this writing the H2O AI Cloud trial environment is not configured for XGBoost
# We will use GLM here
# For XGBoost, see https://github.com/PacktPublishing/Machine-Learning-at-Scale-with-H2O/blob/main/chapt3/local-cluster.ipynb
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
param = {
    "family" : "binomial"
}
model = H2OGeneralizedLinearEstimator(**param)
model.train(
    x = predictors,
    y = "bad_loan",
    training_frame = train,
    validation_frame = validate)

## Evaluate and explain the model

In [ ]:
perf = model.model_performance(test)
perf

In [ ]:
# requires matplotlib
explain = model.explain(test,include_explanations="varimp")
explain

## Export the model scoring artifact

In [ ]:
model.download_mojo("~/download-destination-path") # replace with your local path

## Shut down the cluster

In [ ]:
h2o.cluster().shutdown()